# Class Activation Map (CAM) using Keras

Class Activation Map (CAM) is a powerful technique used in 'Computer Vision' to visualize and get insights from a Convolutional Neural Network (CNN). It is used by scientists to inspect the image to be categorized and it also helps to understand which parts of that image have contributed more to the final output of the model. It creates a heatmap of 'Class Activation' over the given input image.  A 'Class Activation' heatmap is a Two-Dimensional Array of scores associated with a specific output class. The scores are computed for every location in any input image, indicating how important each location is with respect to the class considered. It tells which features the model is looking for.

In this example, I am using Keras to implement CAM because Keras is easy to use and it also saves a lot of time. I will be using a pre-trained CNN, VGG16 model, using weights deriving from its training on the Imagenet dataset.

In [ ]:
from keras.applications.vgg16 import VGG16
from keras import backend as K
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib inline
K.clear_session()

### Loading VGG16 model

In [ ]:
model = VGG16(weights='imagenet')

### Input Image (Sheep)

In [ ]:
img_path = r'read_image/sheep.png'
img=mpimg.imread(img_path)
plt.imshow(img)

### Resizing image to fit the input size of VGG16 model.

In [ ]:
from keras.preprocessing import image
img = image.load_img(img_path, target_size=(224, 224))

### Convert image to numpy array.

In [ ]:
x = image.img_to_array(img)

### Reshaping the data into 'batch form' so that the model could accept it.

In [ ]:
import numpy as np
x = np.expand_dims(x, axis=0)

In [ ]:
x.shape

### Preprocessing

In [ ]:
from keras.applications.vgg16 import preprocess_input
x = preprocess_input(x)

### Predictions

In [ ]:
import pandas as pd
from keras.applications.vgg16 import decode_predictions
preds = model.predict(x)
predictions = pd.DataFrame(decode_predictions(preds, top=3)[0], columns=['column', 'categories', 'probability']).iloc[:, 1:]
print('Prediction:', predictions.loc[0, 'categories'])

In [ ]:
import seaborn as sns
f = sns.barplot(x='probability', y='categories', data=predictions, color='purple')
sns.set_style(style='white')
f.grid(False)
f.spines['top'].set_visible(False)
f.spines['right'].set_visible(False)
f.spines['bottom'].set_visible(False)
f.spines['left'].set_visible(False)
f.set_title('Top 3 Predictions')

### Getting the index of the prediction.

In [ ]:
argmax = np.argmax(preds[0])

In [ ]:
output = model.output[:, argmax]

### Model Archtecture

In [ ]:
model.summary()

In [ ]:
last_conv_layer = model.get_layer('block5_conv3')

In [ ]:
grads = K.gradients(output, last_conv_layer.output)[0]
pooled_grads = K.mean(grads, axis=(0, 1, 2))
iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])
pooled_grads_value, conv_layer_output_value = iterate([x])

In [ ]:
for i in range(512):
    conv_layer_output_value[:, :, i] *= pooled_grads_value[i]

### Plotting the Heatmap.

In [ ]:
heatmap = np.mean(conv_layer_output_value, axis=-1)
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)
plt.matshow(heatmap)
plt.show()

### Load image and resize the heatmap.

In [ ]:
import cv2
img = cv2.imread(img_path)

In [ ]:
heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))

### Convert heatmap into 'RGB' and apply it to original Image.

In [ ]:
heatmap = np.uint8(255 * heatmap)

In [ ]:
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

### Apply heatmap intensity factor.

In [ ]:
hif = .8

In [ ]:
superimposed_img = heatmap * hif + img

### Save to disk and plot.

In [ ]:
output = 'read_image/output_sheep.jpeg'
cv2.imwrite(output, superimposed_img)

img=mpimg.imread(output)

In [ ]:
plt.imshow(img)
plt.axis('off')
plt.title(predictions.loc[0, 'categories'])